<a href="https://colab.research.google.com/github/LivingstonTardzenyuy/Deep-Learning-with-TensorFlow/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences text or speech)

Another common term for NLP problems is sequence to sequence problems(seq2seq).

## Check for GPU

In [19]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-74c7b6a1-79ac-bf1b-f45d-31bb3fd8054e)


## Get helper functions.

In [20]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2025-01-06 02:39:15--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2025-01-06 02:39:15 (15.2 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



## Get our text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as diaster or not diaster).

See the original source here: https://www.kaggle.com/competitions/nlp-getting-started

In [21]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip the data.
unzip_data("nlp_getting_started.zip")

--2025-01-06 02:39:15--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.207, 74.125.195.207, 172.253.117.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2025-01-06 02:39:15 (114 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do that is using Python.

But it's good we get visual straight away. Hence we're going to use pandas.

In [22]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [23]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

# Shuffle training dataFrame.

The reason we shuffle our data is to make sure our model don't have a particular format to learn(so it won't lead to baises).

In [24]:
# Shuffle our training dataset.
train_df_shuffled = train_df.sample(frac=1, random_state=42)   # frac=1 means shuffle all the data.
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [25]:
# What does the test dataFrame look like ?.
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [26]:
## How many examples of each class are they ?.

train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [27]:
# How many total samples ?.
len(train_df), len(test_df)

(7613, 3263)

In [28]:
# Let's visualize some random training examples.
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index: random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
Epicentre - Cydia Tweak - https://t.co/WKmfDig3nT | Thanks to @phillipten.

---

Target: 1 (real disaster)
Text:
Deputies: Dog dispute leads to fatal shooting in Person County http://t.co/OazgHoZGYa #gunfail #nra

---

Target: 0 (not real disaster)
Text:
Happy birthday @lesley_mariiee ?? I miss you so much &amp; i hope you have a great birthday!!????

---

Target: 0 (not real disaster)
Text:
Ngata on injury list at start of practice for Lions http://t.co/Z16DtoQHhG

---

Target: 0 (not real disaster)
Text:
Sometimes you face difficulties not because you're doing something wrong but because you're doing something right. - Joel Osteen

---



### Split data into training and Validation datasets.

In [29]:
from sklearn.model_selection import train_test_split

train_sentence, val_sentence, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),
    train_df_shuffled["target"].to_numpy(),
    test_size=0.1,   # Use 10% for validation data.
    random_state=42
)

# Check length of our sentences.
len(train_sentence), len(train_labels), len(val_sentence), len(val_labels)

(6851, 6851, 762, 762)

In [30]:
# Check out the first 10 samples.
train_sentence[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with text problems, one of the first things you'll have to do before you build a model is to convert your text to numbers.

There are few ways to do this, namely:
* Tokenization: directly mapping a token(a token could be a word or character) to number.
* Embedding - create a matrix of feature vector for each token



```
# This is formatted as code
```

### Text Vectorization(tokenization)

In [38]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
# Use the default text vectorization parameters.
text_vectorizer = TextVectorization(
    max_tokens = 10000,   # how many words in the vocabulary. None means we don't know how many words we can have.
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,   # create groups of n-words?
    output_mode='int',   # how to map tokens to numbers
    output_sequence_length=None,   # how long our do you wnat your sequences to be ?.
    pad_to_max_tokens=True
)

text_vectorizer

<TextVectorization name=text_vectorization_1, built=False>

In [40]:
train_sentence[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [45]:
# Find the average number of tokens(words) in the training tweets
round(sum([len(i.split()) for i in train_sentence]) / len(train_sentence))

15

In [46]:
# Setup text vectorization veriables.
max_vocab_length = 1000  # max number of words to have in our vocabulary.
max_length = 15 # max length our  sequences will be.

text_vectorizer = TextVectorization(
    max_tokens=max_vocab_length,
    output_mode="int",
    output_sequence_length=max_length
)

In [48]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentence)

In [49]:
# Create a sample sentence and tokenize it.
sample_tensence = "Great i'm doing just awersome!"
text_vectorizer([sample_tensence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[196,  32, 487,  29,   1,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [52]:
# Choose randome sentence from the training dataset and tokenize it.
random_sentence = random.choice(train_sentence)
print(f"Original text:\n {random_sentence}\
      \n\n Vectorized version:")
text_vectorizer([random_sentence])

Original text:
 GOP debate drinking game. For anyone looking for a bit of fun while watching this train wreck. http://t.co/W3Rga0nkOm http://t.co/0TZsQe8ESD      

 Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  1,   1,   1, 397,  10, 864, 884,  10,   3,   1,   6, 604, 211,
        636,  19]])>

In [53]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")


Number of words in vocab: 1000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['reported', 'r', 'pray', 'playlist', 'patience']
